### Simultaneous activation of neurons in distinct brain regions

In [1]:
"""
Parse and visualize the stimulus.txt file to understand the learning experiment
"""
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path


def parse_stimulus_file(stimulus_file_path):
    """
    Parse the stimulus.txt file to extract:
    1. Area definitions (which neurons belong to which area)
    2. Stimulation protocol (when/how areas are stimulated)
    
    Returns:
        areas_dict: {area_name: [list of neuron_ids]}
        stimuli: list of dicts with stimulation info
    """
    areas_dict = {}
    stimuli = []
    
    with open(stimulus_file_path, 'r') as f:
        lines = f.readlines()
    
    current_area = None
    
    for line in lines:
        line = line.strip()
        
        # Skip empty lines
        if not line:
            continue
        
        # Check if this is an area definition line
        # Format: "# area_8: us 7071 7072 7073 ..."
        if line.startswith('#') and 'area_' in line.lower():
            parts = line.split(':')
            if len(parts) >= 2:
                # Extract area name (e.g., "8" from "# area_8")
                area_part = parts[0].replace('#', '').strip()
                area_name = area_part.split('_')[-1]  # Get "8" from "area_8"
                
                # Parse neuron IDs after "us" or directly after ":"
                neuron_str = parts[1].strip()
                if neuron_str.startswith('us'):
                    neuron_str = neuron_str[2:].strip()  # Remove "us" prefix
                
                # Parse all neuron IDs (space-separated)
                neuron_ids = [int(x) - 1 for x in neuron_str.split() if x.isdigit()]  # 0-indexed
                
                areas_dict[area_name] = neuron_ids
                current_area = area_name
                print(f"Area {area_name}: {len(neuron_ids)} neurons (IDs: {neuron_ids[0]+1} to {neuron_ids[-1]+1})")
        
        # Check if this is a stimulus definition line
        # Format: <start>-<end>:<frequency> <intensity> <neuron_ids or area reference>
        elif '-' in line and ':' in line and not line.startswith('#'):
            try:
                stimulus_info = parse_stimulus_line(line, areas_dict)
                stimuli.append(stimulus_info)
                print(f"Stimulus: steps {stimulus_info['start']}-{stimulus_info['end']}, "
                      f"frequency {stimulus_info['frequency']}, "
                      f"intensity {stimulus_info['intensity']}, "
                      f"{len(stimulus_info['neurons'])} neurons, "
                      f"areas: {stimulus_info.get('area_names', 'unknown')}")
            except Exception as e:
                print(f"Could not parse stimulus line: {line}")
                print(f"Error: {e}")
    
    return areas_dict, stimuli


def parse_stimulus_line(line, areas_dict):
    """
    Parse a stimulus line:
    Format: <start>-<end>:<frequency> <intensity> <neuron_ids>
    
    Example: "50000-100000:100 1.5 7071 7072 7073 ..."
    """
    # Split by whitespace
    parts = line.split()
    
    # Parse time range and frequency
    time_freq = parts[0]
    time_range, frequency = time_freq.split(':')
    start, end = map(int, time_range.split('-'))
    frequency = int(frequency)
    
    # Parse intensity
    intensity = float(parts[1])
    
    # Parse neuron IDs
    neuron_ids_str = ' '.join(parts[2:])
    neurons = [int(x) - 1 for x in neuron_ids_str.split() if x.isdigit()]  # 0-indexed
    
    # Determine which areas these neurons belong to
    area_names = []
    for area_name, area_neurons in areas_dict.items():
        overlap = len(set(neurons) & set(area_neurons))
        if overlap > 0:
            area_names.append(f"{area_name} ({overlap} neurons)")
    
    return {
        'start': start,
        'end': end,
        'frequency': frequency,
        'intensity': intensity,
        'neurons': neurons,
        'area_names': ', '.join(area_names) if area_names else 'unknown'
    }


def analyze_co_stimulation(stimuli, areas_dict):
    """
    Analyze which areas are co-stimulated (key for understanding learning)
    """
    print("\n=== CO-STIMULATION ANALYSIS ===")
    
    # Group stimuli by time window to find overlapping stimulation
    time_windows = {}
    for stim in stimuli:
        key = (stim['start'], stim['end'])
        if key not in time_windows:
            time_windows[key] = []
        time_windows[key].append(stim)
    
    for time_key, stims in time_windows.items():
        if len(stims) > 1:
            print(f"\nCo-stimulation during steps {time_key[0]}-{time_key[1]}:")
            for stim in stims:
                print(f"  - {stim['area_names']}")
            print("  → These areas should form connections (Hebbian learning)!")
        else:
            print(f"\nSolo stimulation during steps {time_key[0]}-{time_key[1]}:")
            print(f"  - {stims[0]['area_names']}")


def visualize_stimulus_protocol(areas_dict, stimuli, output_dir='stimulus_analysis'):
    """
    Create visualizations showing:
    1. Timeline of stimulation events
    2. Which areas receive stimulation
    """
    os.makedirs(output_dir, exist_ok=True)
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))
    
    # Plot 1: Stimulus timeline with color-coding by area
    area_colors = {
        '8': '#FF6B6B',   # Red
        '30': '#4ECDC4',  # Teal
        '34': '#FFD93D',  # Yellow
    }
    
    y_position = 0
    legend_handles = []
    
    for idx, stim in enumerate(stimuli):
        start = stim['start']
        end = stim['end']
        area_names = stim.get('area_names', '')
        
        # Determine color based on area
        color = 'gray'
        for area in area_colors.keys():
            if f"area {area}" in area_names.lower() or f"Area {area}" in area_names:
                color = area_colors[area]
                break
        
        bar = ax1.barh(y_position, end - start, left=start, height=0.6, 
                color=color, alpha=0.7, edgecolor='black', linewidth=1)
        
        # Add label
        label = area_names.split('(')[0].strip()  # Get just the area name
        ax1.text(start + (end-start)/2, y_position, label, 
                ha='center', va='center', fontsize=9, fontweight='bold')
        
        y_position += 1
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=color, label=f'Area {area}', alpha=0.7) 
                      for area, color in area_colors.items()]
    ax1.legend(handles=legend_elements, loc='upper right', fontsize=11)
    
    ax1.set_xlabel('Simulation Step', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Stimulus Event', fontsize=13, fontweight='bold')
    ax1.set_title('Stimulation Timeline - Learning Experiment', fontsize=15, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)
    ax1.axvline(50000, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Plasticity starts')
    ax1.set_xlim(0, max([s['end'] for s in stimuli]) + 10000)
    
    # Plot 2: Neuron count per area
    area_names_sorted = sorted(areas_dict.keys(), key=lambda x: int(x) if x.isdigit() else 0)
    neuron_counts = [len(areas_dict[a]) for a in area_names_sorted]
    
    colors_bar = [area_colors.get(a, 'gray') for a in area_names_sorted]
    
    bars = ax2.bar(area_names_sorted, neuron_counts, color=colors_bar, 
                   alpha=0.7, edgecolor='black', linewidth=1.5)
    ax2.set_xlabel('Brain Area', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Number of Neurons', fontsize=13, fontweight='bold')
    ax2.set_title('Neuron Distribution by Area', fontsize=15, fontweight='bold')
    ax2.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, area in zip(bars, area_names_sorted):
        height = bar.get_height()
        label = f'{int(height)}'
        if area in ['8', '30', '34']:
            label += '\n(STIM)'  # Mark stimulated areas
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                label, ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'stimulus_protocol.png'), dpi=150, bbox_inches='tight')
    print(f"\n✓ Saved: {output_dir}/stimulus_protocol.png")
    plt.close()
    
    # Print summary
    print("\n" + "="*60)
    print("STIMULUS SUMMARY")
    print("="*60)
    print(f"\nTotal brain areas defined: {len(areas_dict)}")
    for area in area_names_sorted:
        neurons = areas_dict[area]
        marker = " ← STIMULATED" if area in ['8', '30', '34'] else ""
        print(f"  Area {area}: {len(neurons)} neurons{marker}")
    
    print(f"\nTotal stimulus events: {len(stimuli)}")
    for idx, stim in enumerate(stimuli, 1):
        print(f"\n  Event {idx}:")
        print(f"    Time: steps {stim['start']:,} - {stim['end']:,}")
        print(f"    Frequency: every {stim['frequency']} steps")
        print(f"    Intensity: {stim['intensity']}")
        print(f"    Neurons: {len(stim['neurons'])}")
        print(f"    Areas: {stim['area_names']}")



In [2]:

def main():
    """
    Main function to analyze stimulus file
    """
    BASE_DIR = r"C:\Users\A\Desktop\MSc CLS\SVVR\project"
    
    stimulus_file = os.path.join(BASE_DIR, "viz-stimulus", "stimulus.txt")
    
    print("="*60)
    print("LEARNING SIMULATION ANALYSIS")
    print("="*60)
    print(f"\nReading: {stimulus_file}\n")
    
    if not os.path.exists(stimulus_file):
        print(f"❌ ERROR: File not found: {stimulus_file}")
        return
    
    # Parse the file
    areas_dict, stimuli = parse_stimulus_file(stimulus_file)
    
    # Analyze co-stimulation patterns
    if stimuli:
        analyze_co_stimulation(stimuli, areas_dict)
    
    # Create visualizations
    if areas_dict or stimuli:
        visualize_stimulus_protocol(areas_dict, stimuli)
        print("\n" + "="*60)
        print("✓ ANALYSIS COMPLETE!")
        print("="*60)
        print("\nKEY INSIGHT:")
        print("Areas 8 and 30 are co-stimulated → should form connections")
        print("Area 34 is stimulated separately → control group (no learning expected)")
    else:
        print("\n❌ No areas or stimuli found in file.")


if __name__ == "__main__":
    main()

LEARNING SIMULATION ANALYSIS

Reading: C:\Users\A\Desktop\MSc CLS\SVVR\project\viz-stimulus\stimulus.txt

Area 8: 2820 neurons (IDs: 7071 to 31070)
Area 30: 1810 neurons (IDs: 11 to 7070)
Area 34: 1910 neurons (IDs: 4251 to 22810)
Stimulus: steps 150000-152000, frequency 1, intensity 8.4, 2820 neurons, areas: 8 (2820 neurons)
Stimulus: steps 200000-202000, frequency 1, intensity 8.4, 1810 neurons, areas: 30 (1810 neurons)
Stimulus: steps 250000-252000, frequency 1, intensity 8.4, 1910 neurons, areas: 34 (1910 neurons)
Stimulus: steps 300000-302000, frequency 1, intensity 8.4, 4630 neurons, areas: 8 (2820 neurons), 30 (1810 neurons)
Stimulus: steps 350000-352000, frequency 1, intensity 8.4, 1910 neurons, areas: 34 (1910 neurons)
Stimulus: steps 400000-402000, frequency 1, intensity 8.4, 4630 neurons, areas: 8 (2820 neurons), 30 (1810 neurons)
Stimulus: steps 450000-452000, frequency 1, intensity 8.4, 1910 neurons, areas: 34 (1910 neurons)
Stimulus: steps 500000-502000, frequency 1, inte